In [77]:
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import *

from ImageDataAugmentor.image_data_augmentor import *
import albumentations

from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir('/kaggle/input/sfcarclassif'))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

In [79]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS               = 4  # эпох на обучение
BATCH_SIZE           = 16 # уменьшаем batch если сеть большая, иначе не поместится в память на GPU
LR                   = 1e-4
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 224 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '/kaggle/input/sfcarclassif'
PATH = "/kaggle/working" # рабочая директория

RANDOM_SEED = 7

In [80]:
train_df = pd.read_csv(DATA_PATH+"/train.csv")
sample_submission = pd.read_csv(DATA_PATH+"/sample-submission.csv")
train_df.head()

In [81]:
train_df.info()

In [82]:
train_df.Category.value_counts()

In [83]:
image = PIL.Image.open(DATA_PATH+'/train'+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

In [85]:
AUGMENTATIONS = albumentations.Compose([
    albumentations.ShiftScaleRotate(
        p=0.8, #apply these to 80% of the images
        shift_limit=0.1, #translate by (-0.1%, 0.1%)
        scale_limit=0.1, #zoom by (-0.1%, 0.1%)
        rotate_limit=20, #rotate by (-20, 20)
        border_mode=cv2.BORDER_CONSTANT, value = [0,0,0], #fill emptyness with (0,0,0)
        ),
    albumentations.Blur(blur_limit=1, p=0.2),
    albumentations.ElasticTransform(alpha=0.1, sigma=5, alpha_affine=2,
                                     border_mode=cv2.BORDER_CONSTANT, value = [0,0,0], #fill emptyness with (0,0,0)
                                    ),
    albumentations.ToFloat(max_value=255),
])

In [86]:
train_datagen = ImageDataAugmentor(augment=AUGMENTATIONS, seed=RANDOM_SEED)

test_datagen = ImageDataAugmentor(augment=albumentations.ToFloat(max_value=255), seed=RANDOM_SEED)

In [87]:
train_generator = train_datagen.flow_from_directory(
    DATA_PATH+'/train/train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    DATA_PATH+'/train/train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=DATA_PATH+'/test/test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

In [88]:
base_model = Xception(weights='imagenet', include_top=False, input_shape = input_shape)

In [89]:
base_model.summary()

In [90]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [91]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)
# and a logistic layer -- let's say we have 10 classes
predictions = Dense(CLASS_NUM, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=LR), metrics=["accuracy"])

In [92]:
model.summary()

In [93]:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
callbacks_list = [checkpoint]

In [94]:
history = model.fit(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = [learning_rate_reduction]
)

In [95]:
model.save('kaggle/working/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [96]:
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [97]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

In [98]:
test_sub_generator.samples

In [99]:
test_sub_generator.reset()
predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [100]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

In [ ]:
submission.head()